In [1]:
from utils import transcription_utils as utils
import warnings
from IPython.display import Markdown, display

In [2]:

# Disable SSL warnings (optional but not recommended in production)
warnings.filterwarnings("ignore")

In [3]:
# 1. Settings
AUDIO_DIR = "./resources/input/"
OUTPUT_DIR = "./resources/output/"
MODEL_SIZE = "medium"  # You can use tiny, base, small, medium, large
LANGUAGE = "spanish"

summary = utils.transcribe_audio(AUDIO_DIR, OUTPUT_DIR, MODEL_SIZE, LANGUAGE)

display(Markdown(summary))

🎧 Using audio file: ./resources/input\audio_sample.mp3
🧠 Transcribing with Whisper (spanish)... please wait, this can take several minutes...


RuntimeError: Model spanish not found; available models = ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large-v1', 'large-v2', 'large-v3', 'large', 'large-v3-turbo', 'turbo']